In [1]:
#import packages 
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
# below is the qm1 package
from qm1.grid import *
from qm1.operators import *
from qm1.qmsystem import *
from qm1.eigensystem import *
import qm1
print(qm1.__version__)
print(qm1.__file__)


0.0.0
/home/ak/code/btqm1/qm1/__init__.py


In [ ]:
# ------------------------------------------
# simulation grid

# [x_min, x_max] is interval the wave function is defined on
x_min, x_max = -20., 20.             
# number of grid points
num = 500                           
# set boundary condition
# bc = 'open'
bc = 'periodic'
# bc = 'vanishing'
grid = UniformGrid(bc, x_min, x_max, num)

# ------------------------------------------
# set the mass
mass = 1.
# remark:
# the mass is nothing we can manipulate usually. but in changing the mass here,
# some aspects of the time evolution appear more distinct, e.g. the diffusion of a
# electron in free space (no potential).
# correct default value is 1

# ------------------------------------------
# choose from different predefined potentials, e.g.
potential = DoubleDeltaPot(grid)
# or create a sutom potential:
def your_custom_potential(x):
  """user defined potential"""
  your_parameter_1 = np.pi / 15.
  your_parameter_2 = np.sin(18)
  result = np.sin(x/(x_max-x_min)*np.pi)*your_parameter_1 + your_parameter_2
  return result
# potential = your_custom_potential

# ------------------------------------------
# set up the quantum mechanical system
qsys = QMSystem(potential, grid, mass)

In [ ]:
# construct some operators, that act on the wave function 
op_identity = IdentityOp(qsys)
op_position = PositionOp(qsys)
op_momentum = MomentumOp(qsys)
op_hamilton = HamiltonOp(qsys)
op_potential= PotentialOp(qsys)
# make the operators more efficient
make_efficient([op_identity, op_position, op_momentum, op_potential, op_hamilton])

In [ ]:
import pandas
from qm1.eigensystem import Eigensystem

# number of states to calculate
num_states = 10     

# get eigensystem 
eigsys = Eigensystem(num=num_states, operator=op_hamilton)

norms, positionExps, positionVars, momenta, energyExps, energyVars = [], [], [], [], [], []
for _i in range(num_states):
  norms.append(np.real(eigsys.eigstates[_i].expectation_value(op_identity)))
  positionExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_position)))
  positionVars.append(np.real(eigsys.eigstates[_i].variance(op_position)))
  momenta.append(np.real(eigsys.eigstates[_i].expectation_value(op_momentum)))
  energyExps.append(np.real(eigsys.eigstates[_i].expectation_value(op_hamilton)))
  energyVars.append(np.real(eigsys.eigstates[_i].variance(op_hamilton)))

# show the observables
pandas.DataFrame(
  [norms, positionExps, positionVars, momenta, energyExps, energyVars],
  ['norms', 'positionExps', 'positionVars', 'momenta', 'energyExps', 'energyVars'], ['state '+str(_i) for _i in range(num_states)])



In [ ]:

# plot ...
fig, ax1 = plt.subplots(figsize=(15, 5))
fig.subplots_adjust(right=0.6)
colors = [plt.cm.tab10(i) for i in range(num_states)]
alphas = [((num_states-i)/num_states)**2. for i in range(num_states)]

# ... wavefunctions on ax1
ax1.set_title('eigenstates of the hamiltonian')
ax1.set_xlabel('position')
ax1.set_ylabel('wave function')
ax1.set_xlim((qsys.grid.xmin, qsys.grid.xmax))
for _i, (_eigs, _col, _a) in enumerate(zip(eigsys.eigstates, colors, alphas)):
  ax1.plot(qsys.grid.points, _eigs.func, label='state '+str(_i), alpha=_a, color=_col)
ax1.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))

# ... energy levels and potential on ax2
ax2 = ax1.twinx()
ax2.set_ylabel('energy scale / potential')
ax2.plot(qsys.grid.points, qsys.pot, "k")
for _energy, _col, _a in zip(energyExps, colors, alphas):
  ax2.axhline(_energy, color=_col, alpha=_a, ls='--')

plt.plot()
